# Test Environment

In [ ]:
import os
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'platform'

import jax
import jax.numpy as jnp
from envs.aeroplanax_combat import AeroPlanaxCombatEnv, CombatTaskParams

def unbatchify(x: jnp.ndarray, agent_list, num_envs, num_actors):
    x = x.reshape((num_actors, num_envs, -1))
    return {a: x[i] for i, a in enumerate(agent_list)}


rng = jax.random.PRNGKey(0)

# Instantiate the environment & its settings.
env_params = CombatTaskParams()
env = AeroPlanaxCombatEnv(env_params)

num_envs = 100

rng, _rng = jax.random.split(rng)
reset_rng = jax.random.split(_rng, num_envs)
obsv, env_state = jax.vmap(env.reset, in_axes=(0))(reset_rng)

rng, _rng = jax.random.split(rng)
rng_step = jax.random.split(_rng, num_envs)

action = jnp.zeros((num_envs * env.num_agents, 4))

vmap_step = jax.vmap(env.step, in_axes=(0, 0, 0))
obsv, env_state, reward, done, info = vmap_step(
    rng_step, env_state, 
    unbatchify(action, env.agents, num_envs, env.num_agents))

# # Reset the environment.
# key, key_reset = jax.random.split(key)
# obs, state = env.reset(key_reset, env_params)

# Sample a random action.
# for i in range(100):
#     key, key_act, key_step = jax.random.split(key, 3)
#     key_act = jax.random.split(key_act, env.num_agents)
#     actions = {
#         agent: env.action_space(agent, env_params).sample(key_act[i])
#         for i, agent in enumerate(env.agents)
#     }
#     obs, state, reward, done, _ = env.step(key_step, state, actions, env_params)
#     print(f'Time: {state.time}, Done: {done}, Reward: {reward}')

In [ ]:
%timeit vmap_step(rng_step, env_state, unbatchify(action, env.agents, num_envs, env.num_agents))

# Render Environment

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['XLA_PYTHON_MEM_FRACTION'] = '0.7'

import jax
import jax.numpy as jnp
import flax.linen as nn
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import optax
from flax.linen.initializers import constant, orthogonal
import functools
from typing import Sequence, NamedTuple, Any, Dict
from flax.training import checkpoints
from flax.training.train_state import TrainState
import distrax
from envs.wrappers import LogWrapper
from envs.aeroplanax_heading import AeroPlanaxHeadingEnv, HeadingTaskParams
import orbax.checkpoint as ocp


class ScannedRNN(nn.Module):
    @functools.partial(
        nn.scan,
        variable_broadcast="params",
        in_axes=0,
        out_axes=0,
        split_rngs={"params": False},
    )
    @nn.compact
    def __call__(self, carry, x):
        """Applies the module."""
        rnn_state = carry
        ins, resets = x
        rnn_state = jnp.where(
            resets[:, np.newaxis],
            self.initialize_carry(*rnn_state.shape),
            rnn_state,
        )
        new_rnn_state, y = nn.GRUCell(features=ins.shape[1])(rnn_state, ins)
        return new_rnn_state, y

    @staticmethod
    def initialize_carry(batch_size, hidden_size):
        # Use a dummy key since the default state init fn is just zeros.
        cell = nn.GRUCell(features=hidden_size)
        return cell.initialize_carry(jax.random.PRNGKey(0), (batch_size, hidden_size))


class ActorCriticRNN(nn.Module):
    action_dim: Sequence[int]
    config: Dict

    @nn.compact
    def __call__(self, hidden, x):
        if self.config["ACTIVATION"] == "relu":
            activation = nn.relu
        else:
            activation = nn.tanh
        obs, dones = x
        embedding = nn.Dense(
            self.config["FC_DIM_SIZE"], kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(obs)
        embedding = activation(embedding)

        rnn_in = (embedding, dones)
        hidden, embedding = ScannedRNN()(hidden, rnn_in)

        actor_mean = nn.Dense(
            self.config["GRU_HIDDEN_DIM"], kernel_init=orthogonal(2), bias_init=constant(0.0)
        )(embedding)
        actor_mean = activation(actor_mean)
        actor_mean = nn.Dense(
            self.action_dim, kernel_init=orthogonal(0.01), bias_init=constant(0.0)
        )(actor_mean)
        actor_logtstd = self.param("log_std", nn.initializers.zeros, (self.action_dim,))
        pi = distrax.MultivariateNormalDiag(actor_mean, jnp.exp(actor_logtstd))

        critic = nn.Dense(
            self.config["FC_DIM_SIZE"], kernel_init=orthogonal(2), bias_init=constant(0.0)
        )(embedding)
        critic = activation(critic)
        critic = nn.Dense(1, kernel_init=orthogonal(1.0), bias_init=constant(0.0))(
            critic
        )

        return hidden, pi, jnp.squeeze(critic, axis=-1)

class Transition(NamedTuple):
    done: jnp.ndarray
    action: jnp.ndarray
    value: jnp.ndarray
    reward: jnp.ndarray
    log_prob: jnp.ndarray
    obs: jnp.ndarray
    info: jnp.ndarray

def batchify(x: dict, agent_list, num_envs, num_actors):
    x = jnp.stack([x[a] for a in agent_list])
    # print('batchify', x.shape)
    return x.reshape((num_actors * num_envs, -1))


def unbatchify(x: jnp.ndarray, agent_list, num_envs, num_actors):
    x = x.reshape((num_actors, num_envs, -1))
    return {a: x[i] for i, a in enumerate(agent_list)}

def test(config, rng):
    def linear_schedule(count):
        frac = (
            1.0
            - (count // (config["NUM_MINIBATCHES"] * config["UPDATE_EPOCHS"]))
            / config["NUM_UPDATES"]
        )
        return config["LR"] * frac
    # init env
    env_params = HeadingTaskParams()
    env = AeroPlanaxHeadingEnv(env_params)
    env = LogWrapper(env)
    config["NUM_ACTORS"] = env.num_agents

    # init model
    network = ActorCriticRNN(env.action_space(env.agents[0], env_params).shape[0], config=config)
    rng = jax.random.PRNGKey(config['SEED'])
    init_x = (
        jnp.zeros(
            (1, config["NUM_ENVS"] * config["NUM_ACTORS"], *env.observation_space(env.agents[0], env_params).shape)
        ),
        jnp.zeros((1, config["NUM_ENVS"] * config["NUM_ACTORS"])),
    )
    init_hstate = ScannedRNN.initialize_carry(config["NUM_ACTORS"] * config["NUM_ENVS"], config["GRU_HIDDEN_DIM"])
    network_params = network.init(rng, init_hstate, init_x)
    if config["ANNEAL_LR"]:
        tx = optax.chain(
            optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
            optax.adam(learning_rate=linear_schedule, eps=1e-5),
        )
    else:
        tx = optax.chain(
            optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
            optax.adam(config["LR"], eps=1e-5),
        )
    train_state = TrainState.create(
        apply_fn=network.apply,
        params=network_params,
        tx=tx,
    )
    state = {"params": train_state.params, "opt_state": train_state.opt_state, "epoch": jnp.array(0)}
    ckptr = ocp.AsyncCheckpointer(ocp.StandardCheckpointHandler())
    checkpoint = ckptr.restore(config['LOADDIR'], args=ocp.args.StandardRestore(item=state))
    network_params = checkpoint["params"]

    # INIT ENV
    rng, _rng = jax.random.split(rng)
    reset_rng = jax.random.split(_rng, config["NUM_ENVS"])
    obsv, env_state = jax.vmap(env.reset, in_axes=(0))(reset_rng)
    env.render(env_state.env_state, env_params, {'__all__': False}, './tracks/')
    init_hstate = ScannedRNN.initialize_carry(config["NUM_ACTORS"] * config["NUM_ENVS"], config["GRU_HIDDEN_DIM"])

    # TEST LOOP

    def _env_step(test_state):
        env_state, last_obs, last_done, hstate, rng = test_state
        rng, _rng = jax.random.split(rng)
        ac_in = (
            last_obs[np.newaxis, :],
            last_done[np.newaxis, :],
        )
        hstate, pi, value = network.apply(network_params, hstate, ac_in)
        action = pi.sample(seed=_rng)
        log_prob = pi.log_prob(action)
        value, action, log_prob = (
            value.squeeze(0),
            action.squeeze(0),
            log_prob.squeeze(0),
        )

        # STEP ENV
        rng, _rng = jax.random.split(rng)
        rng_step = jax.random.split(_rng, config["NUM_ENVS"])
        obsv, env_state, reward, done, info = jax.vmap(
            env.step, in_axes=(0, 0, 0)
        )(rng_step, env_state, 
            unbatchify(action, env.agents, config["NUM_ENVS"], config["NUM_ACTORS"]))
        env.render(env_state.env_state, env_params, done, './tracks/')
        reward = batchify(reward, env.agents, config["NUM_ENVS"], config["NUM_ACTORS"]).reshape(-1)
        transition = Transition(
            last_done, action, value, reward, log_prob, last_obs, info
        )
        obsv = batchify(obsv, env.agents, config["NUM_ENVS"], config["NUM_ACTORS"])
        done = batchify(done, env.agents, config["NUM_ENVS"], config["NUM_ACTORS"]).reshape(-1)
        test_state = (env_state, obsv, done, hstate, rng)
        return test_state, transition

    rng, _rng = jax.random.split(rng)
    test_state = (
        env_state,
        batchify(obsv, env.agents, config["NUM_ENVS"], config["NUM_ACTORS"]),
        jnp.zeros((config["NUM_ENVS"] * config["NUM_ACTORS"]), dtype=bool),
        init_hstate,
        _rng,
    )
    for _ in range(2000):
        test_state, traj_batch = _env_step(test_state)
        print(f'Time: {test_state[0].env_state.time}, Done: {test_state[2]}, Reward: {traj_batch.reward}')
        
    return {"test_state": test_state, "trajectory": traj_batch}


In [4]:
config = {
    "SEED": 42,
    "LR": 3e-4,
    "NUM_ENVS": 1,
    "NUM_ACTORS": 1,
    "FC_DIM_SIZE": 128,
    "GRU_HIDDEN_DIM": 128,
    "UPDATE_EPOCHS": 16,
    "NUM_MINIBATCHES": 5,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.95,
    "CLIP_EPS": 0.2,
    "ENT_COEF": 1e-3,
    "VF_COEF": 1,
    "MAX_GRAD_NORM": 2,
    "ACTIVATION": "relu",
    "ANNEAL_LR": False,
    "LOADDIR": "/home/xcy/AeroPlanax/results/2025-02-24-04-41/checkpoints/checkpoint_epoch_333" 
}
rng = jax.random.PRNGKey(42)
out = test(config, rng)

Time: [1], Done: [False], Reward: [-0.34836575]
Time: [2], Done: [False], Reward: [-0.34838822]
Time: [3], Done: [False], Reward: [-0.34840775]
Time: [4], Done: [False], Reward: [-0.34842274]
Time: [5], Done: [False], Reward: [-0.34845173]
Time: [6], Done: [False], Reward: [-0.3484944]
Time: [7], Done: [False], Reward: [-0.34854242]
Time: [8], Done: [False], Reward: [-0.34858027]
Time: [9], Done: [False], Reward: [-0.34858385]
Time: [10], Done: [False], Reward: [-0.34859598]
Time: [11], Done: [False], Reward: [-0.34854892]
Time: [12], Done: [False], Reward: [-0.34840512]
Time: [13], Done: [False], Reward: [-0.34813407]
Time: [14], Done: [False], Reward: [-0.34773332]
Time: [15], Done: [False], Reward: [-0.34717953]
Time: [16], Done: [False], Reward: [-0.34651485]
Time: [17], Done: [False], Reward: [-0.3457497]
Time: [18], Done: [False], Reward: [-0.34491435]
Time: [19], Done: [False], Reward: [-0.3440274]
Time: [20], Done: [False], Reward: [-0.3431012]
Time: [21], Done: [False], Reward